In [13]:
# Import necessary libraries
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler
#import openai
from langfuse.openai import openai
from langfuse.decorators import observe
# Load environment variables
load_dotenv()

langfuse_callback_handler = LlamaIndexCallbackHandler()
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

client = openai.Client()

# Load documents from a directory (you can change this path as needed)
documents = SimpleDirectoryReader("data").load_data()

# Create an index from the documents
index = VectorStoreIndex.from_documents(documents)

# Create a retriever to fetch relevant documents
retriever = index.as_retriever(retrieval_mode='similarity', k=3)

query = "What years does the strategic plan cover?"

# Retrieve relevant documents
relevant_docs = retriever.retrieve(query)

print(f"Number of relevant documents: {len(relevant_docs)}")
print("\n" + "="*50 + "\n")

for i, doc in enumerate(relevant_docs):
    print(f"Document {i+1}:")
    print(f"Text sample: {doc.node.get_content()[:200]}...")  # Print first 200 characters
    print(f"Metadata: {doc.node.metadata}")
    print(f"Score: {doc.score}")
    print("\n" + "="*50 + "\n")

model_kwargs = {
    "model": "chatgpt-4o-latest",
    "temperature": 0.3,
    "max_tokens": 500
}

user_query="What years does the strategic plan cover?"

relevant_docs_content = "\n".join([doc.node.get_content() for doc in relevant_docs])

message = relevant_docs_content + "\nUser Query: " + user_query

@observe()
def get_openai_response():
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": message}
        ],
        **model_kwargs
    )
    
    response_content = response.choices[0].message.content
    print(response_content)
    
# Run the  function
get_openai_response()


# Create a query engine
#query_engine = index.as_query_engine()

# Example query
#response = query_engine.query("What years does the strategic plan cover?")

#print(response)

Number of relevant documents: 2


Document 1:
Text sample: CodePath
2024-28
Strategic
Plan
+
Appendices
(V1
|
04.23.24)
Executive
Summary
Problem
Solution
Impact
and
Evidence
To
Date
Introduction
to
the
2024-28
Plan
Pillar
1:
Scale
Nationally
in
Breadth,
and
...
Metadata: {'page_label': '1', 'file_name': 'CodePath strategic plan.pdf', 'file_path': '/Users/deepakdhiman/work/work/workspaces/cpaillp/rag_demo/data/CodePath strategic plan.pdf', 'file_type': 'application/pdf', 'file_size': 1452743, 'creation_date': '2024-09-21', 'last_modified_date': '2024-09-21'}
Score: 0.8307755796376529


Document 2:
Text sample: corporate
and
college
customers.
Given
that
it
takes
time
for
sales
personnel
to
ramp
up,
coupled
with
the
six-
to
nine-month
lead
time
required
to
close
six-figure
enterprise
deals,
this
investment
w...
Metadata: {'page_label': '24', 'file_name': 'CodePath strategic plan.pdf', 'file_path': '/Users/deepakdhiman/work/work/workspaces/cpaillp/rag_demo/data/CodePath strategic plan.pd